In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

# Setup WebDriver
service = Service("/opt/homebrew/Caskroom/chromedriver/125.0.6422.60/chromedriver-mac-arm64/chromedriver")
driver = webdriver.Chrome(service=service)

# Navigate to the login page
driver.get("https://stathead.com/users/login.cgi")  # Update if the login URL differs
print("Please log in to the website. Proceed to the next cell once logged in.")


Please log in to the website. Proceed to the next cell once logged in.


In [27]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


def fetch_csv_data(url):
    driver.get(url)
    wait = WebDriverWait(driver, 10)  # Setup wait for up to 10 seconds

    # Check for "no results" message before proceeding
    no_results_text = "Sorry, there are no results for your search."
    page_content = driver.page_source
    if no_results_text in page_content:
        print("No more results to fetch.")
        return None

    try:
        # Find and hover over the parent element to make the export button visible
        hover_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//li[@class="hasmore"]/span[text()="Export Data"]')))
        ActionChains(driver).move_to_element(hover_element).perform()

        # Now wait for the specific export button to become clickable after the hover
        export_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@class="tooltip" and contains(@tip, "suitable for use with Excel") and text()="Get table as CSV (for Excel)"]')))
        export_button.click()

        # Wait for the CSV data to become visible
        wait.until(EC.visibility_of_element_located((By.ID, 'csv_stats')))
    except TimeoutException as e:
        print("Timeout while waiting for elements:", e)
        return ""
    except Exception as e:
        print("Error interacting with page elements:", e)
        return ""

    # Extract CSV data
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pre_tag = soup.find('pre', id='csv_stats')
    return pre_tag.text if pre_tag else ""

In [13]:
# NBA foreign
# base_url = "https://stathead.com/basketball/player-season-finder.cgi?request=1&draft_pick_type=overall&comp_type=reg&order_by=ws&match=player_season&season_start=1&display_type=totals&season_end=-1&locationMatch=isnot&pob=USA&comp_id=NBA&year_min=1947&offset="
# NBA all
# base_url = "https://stathead.com/basketball/player-season-finder.cgi?request=1&draft_pick_type=overall&comp_type=reg&order_by=ws&match=player_season&season_start=1&year_max=2024&display_type=totals&season_end=-1&comp_id=NBA&year_min=1947&offset="
# NFL all
# base_url = "https://stathead.com/football/player-season-finder.cgi?request=1&draft_pick_type=overall&comp_type=reg&order_by=av&match=player_season&season_start=1&season_end=-1&weight_max=500&rookie=N&year_min=2012&cstat[1]=pass_cmp&ccomp[1]=gt&cval[1]=0&cstat[2]=rush_att&ccomp[2]=gt&cval[2]=0&cstat[3]=targets&ccomp[3]=gt&cval[3]=0&cstat[4]=all_td&ccomp[4]=gt&cval[4]=0&cstat[5]=sacks&ccomp[5]=gt&cval[5]=0&cstat[6]=def_int&ccomp[6]=gt&cval[6]=0&cstat[7]=punt&ccomp[7]=gt&cval[7]=0&cstat[8]=touches&ccomp[8]=gt&cval[8]=0&offset="
# NFL Measurables
base_url = "https://stathead.com/football/player-season-finder.cgi?request=1&draft_pick_type=overall&comp_type=reg&order_by=height&match=player_season_combined&season_start=1&season_end=-1&weight_max=500&rookie=N&offset="
# MLB Batting foreign
# base_url = "https://stathead.com/baseball/player-batting-season-finder.cgi?request=1&draft_pick_type=overall&location=pob&comp_type=reg&order_by=b_war&match=player_season&season_start=1&weight_max=500&season_end=-1&locationMatch=isnot&pob=USA&exactness=anymarked&offset="
# MLB Pitching foreign
# base_url = "https://stathead.com/baseball/player-pitching-season-finder.cgi?request=1&draft_pick_type=overall&location=pob&comp_type=reg&order_by=p_war&match=player_season&season_start=1&p_g=x&weight_max=500&season_end=-1&locationMatch=isnot&pob=USA&offset="
# MLS foreign
# base_url = "https://stathead.com/fbref/player-season-finder.cgi?request=1&height_type=height_feet&force_min_year=1&height_max=84&comp_type=c-22&order_by=plus_minus&match=player_season&per90_type=player&weight_max=500&locationMatch=isnot&pob=USA&phase_id=0&comp_gender=m&per90min_val=5&weight_type=lbs&cstat[1]=assisted_shots&ccomp[1]=gt&cval[1]=0&cstat[2]=tackles_won&ccomp[2]=gt&cval[2]=0&cstat[3]=minutes_per_game&ccomp[3]=gt&cval[3]=0&cstat[4]=fouls&ccomp[4]=gt&cval[4]=0&cstat[5]=on_xg_for&ccomp[5]=gt&cval[5]=0&offset="
# NHL foreign
# base_url = "https://stathead.com/hockey/player-season-finder.cgi?request=1&draft_pick_type=overall&comp_type=reg&order_by=goals&match=player_season&season_start=1&season_end=-1&locationMatch=isnot&pob=USA&rookie=N&pos=S&comp_id=NHL&cstat[1]=ops&ccomp[1]=gt&cval[1]=0&cstat[2]=goals_per_game&ccomp[2]=gt&cval[2]=0&offset="
offset = 0
raw_data_list = []

while True:
    url = f"{base_url}{offset}"
    csv_data = fetch_csv_data(url)
    if csv_data is None or "Sorry, there are no results for your search." in csv_data:
        print("Finished fetching all available data.")
        break
    raw_data_list.append(csv_data)
    offset += 200

# Combine all fetched data into one large string
all_csv_data = "\n".join(raw_data_list)


No more results to fetch.
Finished fetching all available data.


In [14]:
def clean_csv_data(all_csv_data):
    lines = all_csv_data.split('\n')
    cleaned_lines = []
    header_found = False
    for line in lines:
        if 'Rk,Player,' in line and header_found:
            continue  # Skip duplicate headers
        elif '--- When using SR' in line or '</' in line or '<a href' in line:
            continue  # Skip lines containing footer content or HTML tags
        cleaned_lines.append(line)
        if 'Rk,Player,' in line:
            header_found = True  # Mark header as found
    
    return '\n'.join(cleaned_lines)

cleaned_csv = clean_csv_data(all_csv_data)


In [15]:
dataframe = pd.read_csv(StringIO(cleaned_csv))
dataframe

,Rk,Player,Ht.,From,To,Age,G,GS,Ht..1,Wt.,BMI,Pos,Team,Player-additional
0,1,Richard Sligh,7-0,1967,1967,23,8,0.0,7-0,300.0,29.9,DT,OAK,SligRi20
1,2,Morris Stroud,6-10,1969,1974,23-28,69,49.0,6-10,255.0,26.7,TE,KAN,StroMo00
2,3,Demar Dotson,6-9,2009,2020,24-35,138,114.0,6-9,315.0,33.8,T,DENTAM,DotsDe00
3,4,Jared Gaither,6-9,2007,2012,21-26,52,37.0,6-9,350.0,37.5,LT,BALKANSDG,GaitJa20
4,5,Buddy Hardaway,6-9,1978,1978,24,12,0.0,6-9,309.0,33.1,T,SDG,HardBu20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27544,27545,Perce Wilson,0-0,1920,1920,30,7,3.0,0-0,150.0,NaN,BB,DET,WilsPe20
27545,27546,Wood,0-0,1920,1920,NaN,2,1.0,0-0,0.0,NaN,NaN,DET,Woodxx20
27546,27547,Henry Zehrer,0-0,1926,1926,21,7,1.0,0-0,175.0,NaN,NaN,HRT,ZehrHe20
27547,27548,Harold Zerbe,0-0,1926,1926,25,1,0.0,0-0,165.0,NaN,NaN,CAN,ZerbHa20


In [16]:
# filename = 'NBA_all.csv'
# filename = 'NFL_all.csv'
filename = 'NFL_measure.csv'
# filename = 'MLB_B_foreign.csv'
# filename = 'MLB_P_foreign.csv'
# filename = 'MLS_foreign.csv'
# filename = 'NHL_foreign.csv'
dataframe.to_csv(filename, index=False)
print(f"Data saved to {filename}.")


Data saved to NFL_measure.csv.


## NFL Birthplaces

In [50]:
import os
import csv

# Open the main page
main_url = "https://www.pro-football-reference.com/friv/birthplaces.htm"
driver.get(main_url)

# Extract all links from the 'Country' and 'State' columns
elements = driver.find_elements(By.CSS_SELECTOR, "td[data-stat='birth_country'] a, td[data-stat='birth_state'] a")
urls = [element.get_attribute('href') for element in elements]
print(f"Total links found: {len(urls)}")

for url in urls:
    print(f"Extracted link: {url}")



Total links found: 174
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=&state=
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=Unknown&state=FL
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state=
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state=AK
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state=AL
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state=American%20Samoa
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state=AR
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state=AZ
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state=CA
Extracted link: https://www.pro-football-reference.com/friv/birthplaces.cgi?country=USA&state

In [52]:
def fetch_csv_data_nfl_birth(url):
    driver.get(url)
    wait = WebDriverWait(driver, 10)  # Setup wait for up to 10 seconds

    # Check for "no results" message before proceeding
    no_results_text = "Sorry, there are no results for your search."
    page_content = driver.page_source
    if no_results_text in page_content:
        print("No more results to fetch.")
        return None

    try:
        # Hover over the "Share & Export" menu to reveal hidden menu options
        hover_element = wait.until(EC.visibility_of_element_located(
            (By.XPATH, '//li[@class="hasmore"]/span[contains(text(),"Share & Export")]')))
        ActionChains(driver).move_to_element(hover_element).perform()

        # Click on the "Get table as CSV (for Excel)" button
        export_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//button[contains(text(),"Get table as CSV (for Excel)")]')))
        export_button.click()

        # Wait for the CSV data to become visible
        wait.until(EC.visibility_of_element_located((By.ID, 'csv_birthplaces')))
    except TimeoutException as e:
        print("Timeout while waiting for elements:", e)
        return ""
    except Exception as e:
        print("Error interacting with page elements:", e)
        return ""

    # Extract CSV data
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pre_tag = soup.find('pre', id='csv_birthplaces')
    return pre_tag.text if pre_tag else ""

In [53]:
from urllib.parse import urlparse, parse_qs

def create_filename(base_dir, url):
    # Parse the URL to extract query parameters
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)

    # Extract 'country' and 'state' parameters, providing defaults if they are empty
    country = query_params.get('country', ['Unknown'])[0]  # Default to 'Unknown' if missing
    state = query_params.get('state', [''])[0]  # Default to empty string if missing

    # Construct a meaningful filename
    # If 'state' is empty, do not include it in the filename
    if state:
        filename = f"{country}_{state}"
    else:
        filename = f"{country}"

    # Ensure filename does not have illegal characters and is not too long
    filename = "".join([c for c in filename if c.isalnum() or c in (' ', '_', '-')]).rstrip()
    if len(filename) > 255:  # Filename length limit for most filesystems
        filename = filename[:255]

    return os.path.join(base_dir, filename + ".csv")

In [54]:
# Directory to store CSV files
data_dir = "./NFL_birth_data"
os.makedirs(data_dir, exist_ok=True)

# Fetch and save CSV data from each URL
for url in urls:
    csv_data = fetch_csv_data_nfl_birth(url)
    if csv_data:
        cleaned_csv = clean_csv_data(csv_data)
        dataframe = pd.read_csv(StringIO(cleaned_csv))

        # Save to CSV file
        country_state = url.split('=')[-1]  # Simple way to name files uniquely
        filename = create_filename(data_dir, url)
        dataframe.to_csv(filename, index=False)
        print(f"Data saved to {filename}")

Data saved to ./NFL_birth_data/Unknown.csv
Data saved to ./NFL_birth_data/Unknown_FL.csv
Data saved to ./NFL_birth_data/USA.csv
Data saved to ./NFL_birth_data/USA_AK.csv
Data saved to ./NFL_birth_data/USA_AL.csv
Data saved to ./NFL_birth_data/USA_American Samoa.csv
Data saved to ./NFL_birth_data/USA_AR.csv
Data saved to ./NFL_birth_data/USA_AZ.csv
Data saved to ./NFL_birth_data/USA_CA.csv
Data saved to ./NFL_birth_data/USA_CO.csv
Data saved to ./NFL_birth_data/USA_CT.csv
Data saved to ./NFL_birth_data/USA_DC.csv
Data saved to ./NFL_birth_data/USA_DE.csv
Data saved to ./NFL_birth_data/USA_FL.csv
Data saved to ./NFL_birth_data/USA_GA.csv
Data saved to ./NFL_birth_data/USA_Guam.csv
Data saved to ./NFL_birth_data/USA_HI.csv
Data saved to ./NFL_birth_data/USA_IA.csv
Data saved to ./NFL_birth_data/USA_ID.csv
Data saved to ./NFL_birth_data/USA_IL.csv
Data saved to ./NFL_birth_data/USA_IN.csv
Data saved to ./NFL_birth_data/USA_KS.csv
Data saved to ./NFL_birth_data/USA_KY.csv
Data saved to ./NF

In [55]:
# # Combine all CSV files into one
# all_csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
# combined_csv_path = "./NFL_births.csv"
# with open(combined_csv_path, 'w', newline='') as combined_file:
#     csv_writer = csv.writer(combined_file)
#     for file in all_csv_files:
#         with open(file, 'r') as f:
#             csv_reader = csv.reader(f)
#             for row in csv_reader:
#                 csv_writer.writerow(row)

In [59]:
import os
import csv
import country_converter as coco

# Directory where CSV files are stored
data_dir = "./NFL_birth_data"

# Function to parse country and state from the filename
def parse_location_from_filename(filename):
    # Assuming the filename format is 'Country_State.csv' or 'Country.csv'
    parts = filename.replace('.csv', '').split('_')
    country = parts[0]
    state = parts[1] if len(parts) > 1 else ""
    return country, state

# Country converter setup
converter = coco.CountryConverter()

# Combine all CSV files into one
all_csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
combined_csv_path = "./NFL_births.csv"

with open(combined_csv_path, 'w', newline='') as combined_file:
    csv_writer = csv.writer(combined_file)
    
    first_file = True  # Flag to help with writing headers
    city_index = None  # To keep track of where the 'City' column is
    
    for file_path in all_csv_files:
        country, state = parse_location_from_filename(os.path.basename(file_path))
        # Standardize country names
        country_aliases = {
            "Unknown": "United States",
            "Wales": "United Kingdom",
            "Scotland": "United Kingdom",
            "Yugoslavia": "Serbia",
            "Okinawa": "Japan"
        }
        country = country_aliases.get(country, country)
        country_code = converter.convert(names=country, to='ISO3')
        
        with open(file_path, 'r') as f:
            csv_reader = csv.reader(f)
            for i, row in enumerate(csv_reader):
                if i == 0:
                    # Identify the 'City' column index
                    city_index = row.index('City') if 'City' in row else None
                    if first_file:
                        # Write headers in the first file only
                        csv_writer.writerow(row + ['Country', 'State', 'Birth Location'])
                else:
                    # Extract city from the row using the city_index
                    city = row[city_index] if city_index is not None else ""
                    birth_location = f"{city} {country_code}" if city else f"{country_code}"
                    # Write the data row along with new columns
                    csv_writer.writerow(row + [country, state, birth_location])
                    
        first_file = False  # After the first file, set this to False

print("All files have been successfully combined.")


All files have been successfully combined.


In [73]:
import os
import csv
import country_converter as coco

# Directory where CSV files are stored
data_dir = "./NFL_birth_data"

# Function to parse country and state from the filename
def parse_location_from_filename(filename):
    parts = filename.replace('.csv', '').split('_')
    country = parts[0]
    state = parts[1] if len(parts) > 1 else ""
    return country, state

# Define a dictionary mapping countries to their capitals
country_capitals = {
    "BHS": "Nassau",  # Bahamas
    "BRB": "Bridgetown",  # Barbados
    "LBN": "Beirut",  # Lebanon
    "DNK": "Copenhagen",  # Denmark
    "ZWE": "Harar",  # Zimbabwe
    "ITA": "Rome",  # Italy
    "ROU": "Bucharest",  # Romania
    "LTU": "Vilnius",  # Lithuania
    "BLZ": "Belmopan",  # Belize
    "JPN": "Tokyo",  # Japan
    "IRL": "Dublin",  # Ireland
    "HRV": "Zagreb",  # Croatia
    "MEX": "Mexico City",  # Mexico
    "DEU": "Berlin",  # Germany
    "BGR": "Sofia",  # Bulgaria
    "GUY": "Georgetown",  # Guyana
    "CIV": "Yamoussoukro",  # Ivory Coast
    "HTI": "Port-au-Prince",  # Haiti
    "AUT": "Vienna",  # Austria
    "UGA": "Kampala",  # Uganda
    "LVA": "Riga",  # Latvia
    "GRC": "Athens",  # Greece
    "SWE": "Stockholm",  # Sweden
    "USA": "Washington, D.C.",  # United States
    "KOR": "Seoul",  # South Korea
    "HKG": "Hong Kong",  # Hong Kong
    "GBR": "London",  # England
    "CMR": "Yaoundé",  # Cameroon
    "NGA": "Abuja",  # Nigeria
    "TON": "Nukuʻalofa",  # Tonga
    "SYR": "Damascus",  # Syria
    "AGO": "Luanda",  # Angola
    "JAM": "Kingston",  # Jamaica
    "POL": "Warsaw",  # Poland
    "LBR": "Monrovia",  # Liberia
    "RUS": "Moscow",  # Russia
    "GHA": "Accra",  # Ghana
    "WSM": "Apia",  # Western Samoa
    "TWN": "Taipei",  # Taiwan
    "MMR": "Naypyidaw",  # Burma (Myanmar)
    "SLE": "Freetown",  # Sierra Leone
    "SRB": "Belgrade",  # Serbia
    "CAN": "Ottawa",  # Canada
    "TTO": "Port of Spain",  # Trinidad and Tobago
    "GIN": "Conakry"  # Guinea
}

# Country converter setup
converter = coco.CountryConverter()

# Combine all CSV files into one
all_csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
combined_csv_path = "./NFL_births.csv"

with open(combined_csv_path, 'w', newline='') as combined_file:
    csv_writer = csv.writer(combined_file)
    
    first_file = True  # Flag to help with writing headers
    city_index = None  # To keep track of where the 'City' column is
    
    for file_path in all_csv_files:
        country, state = parse_location_from_filename(os.path.basename(file_path))
        country_aliases = {
            "Unknown": "United States",
            "Wales": "United Kingdom",
            "Scotland": "United Kingdom",
            "Yugoslavia": "Serbia",
            "Okinawa": "Japan"
        }
        country = country_aliases.get(country, country)
        country_code = converter.convert(names=country, to='ISO3')
        capital = country_capitals.get(country_code, "")  # Get capital if available

        with open(file_path, 'r') as f:
            csv_reader = csv.reader(f)
            for i, row in enumerate(csv_reader):
                if i == 0:
                    city_index = row.index('City') if 'City' in row else None
                    if first_file:
                        csv_writer.writerow(row + ['Country Code', 'Country', 'State', 'Birth Location'])
                else:
                    city = row[city_index] if city_index is not None and row[city_index] else capital
                    birth_location = f"{city} {country_code}" if city else f"{country_code}"
                    csv_writer.writerow(row + [country_code, country, state, birth_location])
                    
        first_file = False  # Reset flag after the first file

print("All files have been successfully combined.")


All files have been successfully combined.
